In [ ]:
import os, glob

# The HOME of the datas of the project
HOME = "/folder/data/"

# The Path to reach the directories
analyses = os.path.join(HOME+"analyses", "")
fileInterleave = os.path.join(HOME+"fileinterleave", "")

# Reference genome
ref = os.path.join(HOME, "reference_genome.txt")
# resequenced genomes in interleave fastq
genomes = glob.glob(os.path.join(fileInterleave, "*interleave.fastq"))

bwa = "bwa" # In the case of a bwa not in the path (more than one version for example)
samtools = "samtools"  # In the case of a samtools not in the path (more than one version for example)
bcftools = "bcftools"

# Do index of the reference genome, it should be done only one time
#os.system(bwa+" index -a bwtsw "+ref)



for genome in genomes:
    sam = os.path.join(analyses, os.path.split(genome.strip(".fastq")+".sam")[1])
    os.system(bwa+" mem -t 2 -p "+ref+" "+genome+"  > "+sam) # do the mapping

    bam = sam.strip(".sam")+".bam"
    os.system(samtools +" view -bS -q 25 -m 90 -f 2 -o "+bam+" "+sam) # filter mapping and convert output in bam format. Only keep reads with al least 90 base perfectly aligned (option -m)

    sortedBam = bam.strip(".bam")+".sorted"
    os.system(samtools+" sort -@ 2 "+bam+" "+sortedBam) # sort the bam mapping file

    BCF = sortedBam.strip(".sorted")+".bcf"
    os.system(samtools+" mpileup -t DP -t SP -uf "+ref+" "+sortedBam+".bam > "+BCF) # create mpileup file for variant calling

    BCF2 = BCF.strip(".bcf")+".raw.bcf"
    os.system(bcftools+" call -O b -vm -o "+BCF2+" "+BCF) #call variant

    VCF = BCF2.strip(".bcf")+".vcf"
    os.system(bcftools+" view "+BCF2+" | vcfutils.pl varFilter -Q 30 -d 10 > "+VCF) # filter snps for quality and depth


In [ ]:
# The HOME of the datas of the project
HOME = "/folder/data/"

# The Path to reach the directories
analyses = os.path.join(HOME+"analyses", "")

#filter vcf for only snps

import os,glob

var = glob.glob(os.path.join(analyses, "*.vcf"))
for elt in var:
    VCFsnps = elt.strip(".vcf")+".snps.vcf"
    cmd = "grep -v \"INDEL\" "+elt+" > "+VCFsnps+" "
    os.system(cmd)


In [ ]:
# filter vcf file to maintain only homozygote variants

lis = glob.glob(os.path.join(analyses, "*.snps.vcf"))

for elem in lis:
    VCFsnpshomo = elem.strip(".vcf")+".homo.vcf"
    o = open(VCFsnpshomo, "w")
    for line in open(elem):
        if len(line.split("\t")) < 5: o.write(line)
        elif "," not in line.split("\t")[4]:
            o.write(line)
    o.close()


In [ ]:
# Compare CLC and Samtools SNPs using bedtools intersect


clc_result =  os.path.join(HOME, "CLC") # the clc vcf files need to be stored in a directory called CLC
samtools_results = os.path.join(HOME, "analyses") #path to the samtools vcf files

genomes = glob.glob(os.path.join(samtools_results, "*.snps.homo.vcf"))

for gen in genomes:
    id = gen.split(".")[0].split("/")[-1]
    print id
    clc = os.path.join(clc_result, ""+id+".clc.vcf") # the name of the clc vcf files need to be: isolate.clc.vcf
    intersect = os.path.join(samtools_results, ""+id+".intersect.vcf")
    os.system("bedtools intersect -a "+gen+" -b "+clc+" > "+intersect) # generate vcf intersect file

    with open(intersect, "r+") as f:     # add format vcf to the head of the file for bedtools intersect
     old = f.read()
     f.seek(0)
     f.write("##fileformat=VCFv4.2\n" + old)


In [ ]:
# Annotations of snps with snpEff

# annotate SNP and keep only SNP in genes
# Do not show DOWNSTREAM changes
# -no-intergenic    Do not show INTERGENIC changes
# -no-intron        Do not show INTRON changes
# -no-upstream        Do not show UPSTREAM changes
# -no-utr        Do not show 5_PRIME_UTR or 3_PRIME_UTR changes

import os

HOME = "/folder/data/"

samtools_results = os.path.join(HOME, "analyses") #path to the samtools vcf files
annotedFiles = os.path.join(HOME, "annotatedfiles")
RE = os.path.join(annotedFiles, "RE.bed")
intersect = glob.glob(os.path.join(samtools_results, "*.intersect.vcf"))



for gen in intersect:
    id = gen.split(".")[0].split("/")[-1]
    eff = gen.strip(".vcf")+".eff.vcf"
    cmd = "java -Xmx4g -jar /folder/snpEff/snpEff.jar ann -no-downstream -no-upstream -interval "+RE+" -v Oidma1 "+gen+" > "+eff+" "
    os.system(cmd)




In [ ]:
%%bash

# Table for general stats mapping and variants


#cd /folder/data/fileInterleave


#for ELEM in *interleave.fastq
#do
#    echo $ELEM
#    echo "Number of reads"
    # Each read of a fastq have 4 lines so we divide by 4 to have the number of reads
#    echo $(wc -l $ELEM) | awk '{split($0,a," "); print a[1] / 4'} | bc
#print echo

#done

# count number of mapped reads
#cd /folder/data


for ELEM in *sorted.bam
# for ELEM in *interleave.bam
do
    echo "Number of mapped reads: " $ELEM

    samtools view -c $ELEM

  print echo

done

# count number of variants
for ELEM in *intersect.vcf
for ELEM in *raw.vcf
do
    echo "Number of variant:" $ELEM
    grep -v "#" $ELEM | wc -l
    #echo "Number of SNPs"$ELEM
    #grep -v "#" $ELEM | grep -v "INDEL" | wc -l
    #echo "Number of INDEL"$ELEM
    #grep -v "#" $ELEM | grep -c "INDEL"

done

# count number of SNPs homozygote
for ELEM in *snps.homo.vcf
do
    echo "Number of homo SNPs:" $ELEM
    grep -v "#" $ELEM | wc -l



done


In [ ]:
import os, glob


##INFO=<ID=ANN,Number=.,Type=String,Description="Functional annotations: 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO' ">


analyses = "folder/data/analyses"

eff = glob.glob(os.path.join(analyses, "*.eff.vcf"))

#java -jar folder/data/snpEff/SnpSift.jar filter "ANN[*].BIOTYPE has 'non_coding'" Zn.intersect.eff.vcf > Zn.non_coding.vcf


for gen in eff:
    id = gen.split(".")[0].split("/")[-1]
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'custom'" """ #count RE
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'intergenic_region'" """ #count intergenic_region
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'missense_variant'" """ #count missense_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'synonymous_variant'" """ #count synonymous_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'stop_gained'" """ #count stop_gained
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'start_lost'" """ #count start_lost
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'stop_lost'" """ #count stop_lost
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'intron_variant'" """ #count intron_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has '5_prime_UTR_variant'" """ #count 5_prime_UTR_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has '3_prime_UTR_variant'" """ #count 3_prime_UTR_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'splice_region_variant'" """ #count splice_region_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].FEATURE has 'transcript'" """ #count transcript
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].BIOTYPE has 'Coding'" """ #count Coding
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'HIGH'" """ #count high impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'MODERATE'" """ #count moderate impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'MODIFIER'" """ #count modifier impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'LOW'" """ #count low impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ERRORS has 'WARNING_TRANSCRIPT_NO_STOP_CODON'" """ #count errors on stop codons
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ERRORS has 'WARNING_TRANSCRIPT_NO_START_CODON'" """ #count errors on start codons
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ERRORS has 'WARNING_TRANSCRIPT_INCOMPLETE'" """ #count errors on transcription
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].ANNOTATION has 'splice_region_variant') ! (ANN[*].ANNOTATION has 'intron') " """ #count splice_region_variant NON intron
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENEID has 'gene_6860') && (ANN[*].FEATURE has 'transcript') " """ #count  transcript gene_6860 Cu/Zn_SOD
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENE has 'jgi.p_Oidma1_18612') && (ANN[*].ANNOTATION has '3_prime_UTR_variant' ) " """ #count 3'UTR gene_6860
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENE has 'jgi.p_Oidma1_18612') && (ANN[*].ANNOTATION has '5_prime_UTR_variant' ) " """ #count 5'UTR gene_6860
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENE has 'jgi.p_Oidma1_18612') && (ANN[*].ANNOTATION has 'intron_variant' ) " """ #count introns gene_6860
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].ANNOTATION has 'intergenic_region') && (ANN[*].GENE has 'jgi.p_Oidma1_18612-jgi.p_Oidma1_53214') " """ #count 5' intergenic_region of gene 6860
    fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].ANNOTATION has 'intergenic_region') && (ANN[*].GENE has 'jgi.p_Oidma1_103006-jgi.p_Oidma1_18612') " """ #count 3' intergenic_region of gene 6860
    cmd = " "+fct+" "+gen+" | grep -v \"#\" | wc -l"

    os.system(cmd)
    print id








In [ ]:
#count snps common to all isolate and total number of snps (with no duplicate)

#importa tutti i path necessari e definisce le variabili
import re,glob, os, sys


HOME = "/folder/data/"
analyses = os.path.join(HOME+"analyses", "")
joinedVcf = os.path.join(analyses, "vcfGenomes")

snps = glob.glob(os.path.join(analyses, "*.intersect.vcf"))


dictSet={}
dictotal = {}

#from the files .intersect.vcf of each isolate, find all the SNPs and identify its position (scaffoldID+position)
#generate a "dicSect" per each isolate
for isolate in snps:
    id = isolate.split("/")[-1].split(".")[0]
    dictSet[id]=set()
    lireFile = open(isolate)
    for line in lireFile.readlines():
        if "#" not in line:
            elts = line.split("\t")
            scaffId= elts[0].split()[0]
            dictSet[id].add(scaffId+"-"+elts[1])
            pos = scaffId+"-"+elts[1]
            dictSet[id].add(pos)
            dictotal[pos] = line

#make intersection and union of all the dictSect and generate two files, respectively "partage" and "ensemble"
partage = set.intersection(dictSet["4H1"],dictSet["5L3"],dictSet["A"],dictSet["E"], dictSet["91"], dictSet["534"], dictSet["505"], dictSet["506"], dictSet["518"], dictSet["523"], dictSet["539"], dictSet["13G"], dictSet["4E"], dictSet["1354"], dictSet["1357"], dictSet["1358"])
ensemble = set.union(dictSet["Cd"], dictSet["4H1"],dictSet["5L3"],dictSet["A"],dictSet["E"], dictSet["91"], dictSet["534"], dictSet["505"], dictSet["506"], dictSet["518"], dictSet["523"], dictSet["539"], dictSet["13G"], dictSet["4E"], dictSet["1354"], dictSet["1357"], dictSet["1358"])



In [ ]:
a = len(dictotal)
b= len(ensemble)
c= len(partage)
print a, b, c

In [ ]:
#script improved (more rapid) to obtain the two files with shared (partage) and total (union) SNPs.

HOME = "/folder/data/"
analyses = os.path.join(HOME+"analyses", "")
joinedVcf = os.path.join(analyses, "vcfGenomes")

outfile = os.path.join(joinedVcf, "common.snps.vcf")
o = open(outfile, "w")

#for each key and value of the dictionary "dictotal" and for each element in "partage" if a correspondence is found, write the values in the output file.
for elt in partage:
    a = dictotal.get(elt)
    o.write(a)

o.close()




#######associates to the "enseble" file with the SNPs their respective annotation

outfile = os.path.join(joinedVcf, "all.snps.vcf")
o = open(outfile, "w")

#for each key and value of the dictionary "dictotal" and for each element in "partage" if a correspondence is found, write the values in the output file.

for elt in ensemble:
    a = dictotal.get(elt)
    o.write(a)

o.close()

In [ ]:
# Annotations of  all (from "ensemble")  and common (from "partage") snps with snpEff

# annotate SNP and keep only SNP in genes
# Do not show DOWNSTREAM changes
# -no-intergenic    Do not show INTERGENIC changes
# -no-intron        Do not show INTRON changes
# -no-upstream        Do not show UPSTREAM changes
# -no-utr        Do not show 5_PRIME_UTR or 3_PRIME_UTR changes

import os, glob

HOME = "/folder/data/"

samtools_results = os.path.join(HOME, "analyses/vcfGenomes") #path to the samtools vcf files
annotedFiles = os.path.join(HOME, "annotatedfiles")
RE = os.path.join(annotedFiles, "RE.bed")
all_common = glob.glob(os.path.join(samtools_results, "*.snps.vcf"))

for gen in all_common:
    id = gen.split(".")[0].split("/")[-1]
    print id
    eff = gen.strip(".snps.vcf")+".eff.vcf"
    cmd = "java -Xmx4g -jar /folder/data/snpEff/snpEff.jar ann -no-downstream -no-upstream -interval "+RE+" -v Oidma1 "+gen+" > "+eff+" "
    os.system(cmd)

In [ ]:
#mappare SNPs totali  e comuni nel genoma

import os, glob


##INFO=<ID=ANN,Number=.,Type=String,Description="Functional annotations: 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO' ">


HOME = "/folder/data/"

eff = glob.glob(os.path.join(HOME, "*.eff.vcf"))

#java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].BIOTYPE has 'non_coding'" Zn.intersect.eff.vcf > Zn.non_coding.vcf


for gen in eff:
    id = gen.split(".")[0].split("/")[-1]
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'custom'" """ #count RE
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'intergenic_region'" """ #count intergenic_region
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'missense_variant'" """ #count missense_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'synonymous_variant'" """ #count synonymous_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'stop_gained'" """ #count stop_gained
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'start_lost'" """ #count start_lost
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'stop_lost'" """ #count stop_lost
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'intron_variant'" """ #count intron_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has '5_prime_UTR_variant'" """ #count 5_prime_UTR_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has '3_prime_UTR_variant'" """ #count 3_prime_UTR_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ANNOTATION has 'splice_region_variant'" """ #count splice_region_variant
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].FEATURE has 'transcript'" """ #count transcript
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].BIOTYPE has 'Coding'" """ #count Coding
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'HIGH'" """ #count high impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'MODERATE'" """ #count moderate impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'MODIFIER'" """ #count modifier impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].IMPACT has 'LOW'" """ #count low impact variants
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ERRORS has 'WARNING_TRANSCRIPT_NO_STOP_CODON'" """ #count errors on stop codons
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ERRORS has 'WARNING_TRANSCRIPT_NO_START_CODON'" """ #count errors on start codons
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "ANN[*].ERRORS has 'WARNING_TRANSCRIPT_INCOMPLETE'" """ #count errors on transcription
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].ANNOTATION has 'splice_region_variant') ! (ANN[*].ANNOTATION has 'intron') " """ #count splice_region_variant NON intron
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENEID has 'gene_6860') && (ANN[*].FEATURE has 'transcript') " """ #count  transcript gene_6860 Cu/Zn_SOD
    fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENE has 'jgi.p_Oidma1_18612') && (ANN[*].ANNOTATION has '3_prime_UTR_variant' ) " """ #count 3'UTR gene_6860
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENE has 'jgi.p_Oidma1_18612') && (ANN[*].ANNOTATION has '5_prime_UTR_variant' ) " """ #count 5'UTR gene_6860
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].GENE has 'jgi.p_Oidma1_18612') && (ANN[*].ANNOTATION has 'intron_variant' ) " """ #count introns gene_6860
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].ANNOTATION has 'intergenic_region') && (ANN[*].GENE has 'jgi.p_Oidma1_18612-jgi.p_Oidma1_53214') " """ #count 5' intergenic_region of gene 6860
    #fct = """java -jar /folder/data/snpEff/SnpSift.jar filter "(ANN[*].ANNOTATION has 'intergenic_region') && (ANN[*].GENE has 'jgi.p_Oidma1_103006-jgi.p_Oidma1_18612') " """ #count 3' intergenic_region of gene 6860
    cmd = " "+fct+" "+gen+" | grep -v \"#\" | wc -l"

    os.system(cmd)
    print id


In [ ]:
# generate new file fasta for each genome considering snps


import os, glob

#The HOME of the datas of the project
HOME = "/folder/data/"

#The Path to reach the directories
analyses = os.path.join(HOME+"analyses", "")


# Reference genome
ref = os.path.join(HOME, "ref_genome_Oidma_jgi/Oidma1.txt")

#vcf file location
#c = "gunzip *.gz"
#os.system(c)
#print c

vcf = glob.glob(os.path.join(analyses, "*.intersect.vcf"))



for gen in vcf:
  #  c = "export PERL5LIB=/Users/cmurat/Downloads/vcftools_0.1.12b/perl"
  #  os.system(c)
    id = gen.split("/")[-1].split(".")[0]
    newFasta = os.path.join(analyses, ""+id+".new.fasta")
    bamfile = os.path.join(analyses, ""+id+".interleave.sorted.bam")
    perbaseLowCov = os.path.join(analyses, ""+id+".perbaseLowCov.bed")
    mask = os.path.join(analyses, ""+id+".new.mask.fasta")
    snpsFasta = os.path.join(analyses, "toto.snps.fasta")

    cmd1 = "bgzip "+gen+" ; tabix -p vcf "+gen+".gz" # prepare the vcf file with bzip and tabix for vcftools
    cmd2 = "cat "+ref+" | vcf-consensus "+gen+".gz > "+newFasta+"" #generate the fasta file for each genome with snps
    cmd3 = "bedtools genomecov -bga -ibam "+bamfile+" | awk '$4<10' > "+perbaseLowCov+"" #generate bed file with interval with low coverage (in this case < 10 reads)
    cmd4 = "bedtools maskfasta -mc X -fi "+newFasta+" -bed "+perbaseLowCov+" -fo "+mask+" "


    print cmd1
    os.system(cmd1)
    print cmd2
    os.system(cmd2)
    print cmd3
    os.system(cmd3)
    print cmd4
    os.system(cmd4)

In [ ]:
#generate a unique fasta file with snps positions for all genomes using a sorted vcf file!

import os, glob

HOME = "/folder/data/"
analyses = os.path.join(HOME+"analyses", "")

#vcf file location
vcf = glob.glob(os.path.join(analyses, "*.intersect.vcf"))
#vcf = glob.glob(os.path.join(analyses, "505.intersect.vcf")) ##example
total = os.path.join(analyses+"/vcfGenomes", "all.snps.sorted.vcf")

outfile = os.path.join(analyses, "Allsnps.sorted.fasta")
#outfile = os.path.join(analyses, "505snps.sorted.fasta") ##example

out = open(outfile, "a")

for gen in vcf:
    id = gen.split("/")[-1].split(".")[0]
    print gen
    #snpsFasta = os.path.join(analyses, ""+id+".snps2.fasta")
    snpsFasta = os.path.join(analyses, ""+id+".snps.fasta")
    mask = os.path.join(analyses, ""+id+".new.mask.fasta")

    cmd1 = "bedtools getfasta -fi "+mask+" -bed "+total+" -fo  "+snpsFasta+" "
    #print cmd1
    os.system(cmd1)
    a = open(snpsFasta).readlines()
    print id
    out.write("\n>"+id+"\n")
    for line in a:
        if ">" not in line:
            base = line.strip()
            out.write(base)
out.close()

In [ ]:
/usr/bin/bash

cd /folder/data/analyses/

mkdir phylogeny

cat Allsnps.fasta Zn.Allsnps.fasta > phylogeny/Allsnps.def.fasta

SyntaxError: invalid syntax (<ipython-input-23-0759dff019ef>, line 1)

In [ ]:
#count line length
import os

HOME = "folder/data/"
analyses = os.path.join(HOME+"analyses/phylogeny", "")


outfile = open(os.path.join(analyses, "Allsnps.def.fasta"))

for line in outfile:
    if ">" not in line:
        print len(line)


In [ ]:
#count bases

import os

HOME = "folder/data/"
analyses = os.path.join(HOME+"analyses/", "")

outfile = open(os.path.join(analyses, "5L3.new.mask.fasta"))
i = 0
for line in outfile:
    if ">" not in line:
        i = i + len(line)

print i

In [ ]:
#command line for quicktree. Needs bioscript.convert

quicktree -in a -out t -boot 100 Tuofileinstockolmformat > fileout.st.aln.100boot.nwk

https://pypi.python.org/pypi/bioscripts.convert/0.4

In [ ]:
import sys,os,re

from Bio import SeqIO

HOME = "/folder/data/analyses"

fileSeq = os.path.join(HOME, "Allsnps.sorted.fasta")
#fileSeq = os.path.join(HOME+"/phylogeny/", "example.fasta")
fileSeq = open(fileSeq,"r")

outfile = os.path.join(HOME, "Allsnps.sorted.pcadapt")
#outfile = os.path.join(HOME+"/phylogeny/", "example.pcadapt")
outfile = open(outfile, "w")


for cur_record in SeqIO.parse(fileSeq, "fasta") :
    print cur_record.id
    if cur_record.id == "Zn":
        ref = cur_record.seq
    for i in range(len(cur_record.seq)):
        if cur_record.seq[i] == ref[i]:
            outfile.write("0 ")

        elif cur_record.seq[i] == "N" or cur_record.seq[i] == "X":
            outfile.write("9 ")

        else:
            outfile.write("1 ")

    outfile.write("\n")

outfile.close()

In [ ]:
#export 10Kb windows in single files containing the same window of all isolates.
HOME = "folder/data/"

import sys,os,re, glob

from Bio import SeqIO

c

fileSeq = glob.glob(os.path.join(HOME, "*10Kb.fasta"))


for f in fileSeq:
    isolate = f.split(".")[0].split("/")[-1]

    f = open(f,"r")

    for cur_record in SeqIO.parse(f, "fasta") :


        interval = cur_record.id

        fileout = os.path.join(HOME, interval+".fasta")
        fileout = open(fileout, "a")
        fileout.write(">"+isolate+"\n"+str(cur_record.seq)+"\n")
        fileout.close()

In [ ]:
#export each gene in single files containing the sequence of the same gene of all isolates.

HOME = "folder/data/"
import sys,os,re, glob

from Bio import SeqIO

c

fileSeq = glob.glob(os.path.join(HOME, "*genes.fasta"))


for f in fileSeq:
    isolate = f.split(".")[0].split("/")[-1]

    f = open(f,"r")

    for cur_record in SeqIO.parse(f, "fasta") :


        interval = cur_record.id

        fileout = os.path.join(HOME, interval+".fasta")
        fileout = open(fileout, "a")
        fileout.write(">"+isolate+"\n"+str(cur_record.seq)+"\n")
        fileout.close()

In [ ]:
#egglib check format

import egglib

HOME = "folder/data/gene_windows"

align = egglib.Align(HOME+"scaffold_1:25551-27296.fasta", groups=True)
for name,sequence,group in align:
    print 'name:',name, 'group:',group


In [ ]:
#egglib 3.0.0b25 analyses: polymorphism of genes or 10Kb windows
#torino, OTTOBRE 2019
#analisi di polimorfismo sui geni


#####the analysis requires that in at least 6 isolates there are nucleotide in all the positions.


import sys,os,re,glob
import egglib


HOME = "folder/data/"
outfile = os.path.join(HOME+"/genes_analyses", "stats.egglib_genes.txt")
outfile = open(outfile, "w")

outfile.write("Interval\tNum_samples\tNum_sites\tS\tTajima'sD\tTheta\tPi\tFst\tKst\tGst\tWCst\n")


#list of loci to analyse
loci = glob.glob(os.path.join(HOME+"/gene_windows", "*.fasta"))
#loci = glob.glob(os.path.join(HOME+"/10Kbwindows", "*.fasta"))


print len(loci)

cs_all = egglib.stats.ComputeStats()
cs_all.add_stats('Fst', 'nseff', 'lseff', 'WCst', 'D', 'Pi', 'thetaW', 'D', 'S', 'Kst', 'Gst')

cs10 = egglib.stats.ComputeStats()
cs10.add_stats('ns_site')

cs20 = egglib.stats.ComputeStats()
cs20.add_stats('ns_site')

# redo requiring >= MINI sample per population
MINI = 6
for fname in sorted(loci):
    aln = egglib.io.from_fasta(fname, labels=True, alphabet=egglib.alphabets.Alphabet('char', 'ACGT', 'XN-'))
    mapping = aln.group_mapping()
    struct_all = egglib.stats.get_structure(aln, lvl_pop=1, ploidy = 1)
    #print struct_all.as_dict()

    # make two structure objects
    struct_10 = struct_all.as_dict()
    del struct_10[0][0][20]
    struct_10 = egglib.stats.make_structure(* struct_10)
    #print struct_10.as_dict()
    struct_20 = struct_all.as_dict()
    del struct_20[0][0][10]
    struct_20 = egglib.stats.make_structure(* struct_20)
    #print struct_20.as_dict()

    cs_all.configure(struct=struct_all, multi=True)
    cs10.configure(struct=struct_10)
    cs20.configure(struct=struct_20)

    # analyse only sites with enough samples in each pop
    for pos in xrange(aln.ls):
        site = egglib.stats.site_from_align(aln, pos)
        n10 = cs10.process_site(site)['ns_site']
        #print n10
        n20 = cs20.process_site(site)['ns_site']
        #print n20
        if n10 is not None and n10 >= MINI and n20 is not None and n20 >= MINI:
            cs_all.process_site(site)

    stats = cs_all.results()
    #print fname, stats['nseff'], stats['lseff'], stats['S'], stats['D'], stats['thetaW']/stats['lseff'], stats['Pi']/stats['lseff'], stats['Fst'], stats['Kst'], stats['Gst'], stats['WCst']
    #print stats['S']


    # extracts statistics in table
    l= str(fname)
    interval = l.split("/")[-1]
    a, b, c = re.match('scaffold_(\d+)\:(\d+)\-(\d+)\.fasta', interval).groups()
    interval = 'scaffold_{0}:{1}-{2}.fasta'.format(a.rjust(2, '0'), b.rjust(7, '0'), c.rjust(7, '0'))
    F = format(stats['Fst'])
    W = format(stats['WCst'])
    T = format(stats['D'])
    P = format(stats['Pi'])
    Th = format(stats['thetaW'])
    S = format(stats['S'])
    L = format(stats['lseff'])
    N = format(stats['nseff'])
    K = format(stats['Kst'])
    G = format(stats['Gst'])




    #print interval, F, T, P, Th
    outfile.write(interval+"\t"+N+"\t"+L+"\t"+S+"\t"+T+"\t"+Th+"\t"+P+"\t"+F+"\t"+K+"\t"+G+"\t"+W+"\n")

outfile.close()
